In [ ]:
pip install darts


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.8/165.8 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 917.3/917.3 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.1/540.1 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.7/196.7 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.2/869.2 kB 6.7 MB/s eta 0:00:

In [ ]:
!pip install dask[dataframe]

INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.9/241.9 kB 6.4 MB/s eta 0:00:00


In [ ]:

!pip install --upgrade darts

In [ ]:
import pandas as pd
import numpy as np
from darts import TimeSeries
from google.colab import drive
from darts.utils.missing_values import fill_missing_values
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from darts.metrics import rmse, mape, r2_score, mae

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dfr = pd.read_excel(r'/content/drive/My Drive/NVDA.xlsx')

In [ ]:
df=dfr.copy()

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])


In [ ]:

full_date_range = pd.date_range(start=df['Date'].min(), end=df['Date'].max(), freq='D')
df.set_index('Date', inplace=True)
df = df.reindex(full_date_range)

df.reset_index(inplace=True)
df.rename(columns={'index': 'Date'}, inplace=True)

In [ ]:
for i in df.columns[1:]:
  df[i] = df[i].interpolate()

In [ ]:
df.set_index('Date', inplace=True)

In [ ]:
ts_df=df.copy()

In [ ]:
ts_series = TimeSeries.from_dataframe(ts_df[['Close']])

In [ ]:
train, test = ts_series.split_before(pd.Timestamp('2024-03-21'))

In [ ]:
def output(mod):

  def generate_forecast(training_data):
    model_instance = mod
    model_instance.fit(training_data)
    predicted_values = model_instance.predict(n=len(test))
    return predicted_values[0].values()[0]

  training_history = train
  forecasted_values = []

  for time_point in range(len(test)):
      predicted_value = generate_forecast(training_history)
      forecasted_values.append(predicted_value)

      actual_observation = test[time_point]
      training_history = training_history.append(actual_observation)


  predictions_df = pd.DataFrame({
    'Date': test.time_index,
    'Predicted':   forecasted_values

  })
  predictions_df['Date'] = pd.to_datetime(predictions_df['Date'])
  predictions_df.set_index('Date', inplace=True)
  predictions_df['Predicted'] = predictions_df['Predicted'].astype(float)

  pred=TimeSeries.from_dataframe(predictions_df, value_cols='Predicted')
  rmseval =rmse(test, pred)
  mapeval=mape(test, pred)
  r2=r2_score(test,pred)
  maeval=mae(test, pred)

  fig = go.Figure()

  fig.add_trace(go.Scatter(
      x=ts_df.index,
      y=ts_df['Close'],
      mode='lines',
      name='Actual',
      line=dict(color='blue')
  ))

  # Plot predicted values
  fig.add_trace(go.Scatter(
      x=predictions_df.index,
      y=predictions_df['Predicted'],
      mode='lines',
      name='Predicted',
      line=dict(color='red', dash='dash')
  ))

  # Update layout
  fig.update_layout(
      title='LR Predictions vs Actual Values',
      xaxis_title='Date',
      yaxis_title='Stock Price',
      legend_title='Legend',
      template='plotly_white'
  )



  print("RMSE:", rmseval, "MAPE:", mapeval, "R2:", r2, "MAE:", maeval )
  fig.show()


# **Linear Regression**

In [ ]:
from darts.models import LinearRegressionModel

In [ ]:
output(LinearRegressionModel(lags=7))

RMSE: 3.0852847758856874 MAPE: 2.0056208831817735 R2: 0.9663411296562049 MAE: 2.1349988901216026


# **ARIMA**

In [ ]:
from darts.models import ARIMA

In [ ]:
output(ARIMA(p=5, d=1, q=0))

RMSE: 3.063417875069315 MAPE: 1.9661585187469868 R2: 0.9668165521766175 MAE: 2.0919567714760716


# **PROPHET**

In [ ]:
from darts.models import Prophet

In [ ]:
output(Prophet())

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpp24mnve1/62skgief.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpp24mnve1/v5r3v34g.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=72526', 'data', 'file=/tmp/tmpp24mnve1/62skgief.json', 'init=/tmp/tmpp24mnve1/v5r3v34g.json', 'output', 'file=/tmp/tmpp24mnve1/prophet_modelgnt1si0r/prophet_model-20240914180953.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
18:09:53 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
18:09:54 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpp2

RMSE: 9.853415997744287 MAPE: 7.063242318157004 R2: 0.6566929984007871 MAE: 7.739310509719637


# **Random Forest**

In [ ]:
from darts.models import RandomForest

In [ ]:
output(RandomForest(7))

RMSE: 3.7488251528873957 MAPE: 2.5511464373811634 R2: 0.9503065219476318 MAE: 2.718692071606362


# **XGBoost**

In [ ]:
from darts.models import XGBModel

In [ ]:
output(XGBModel(7))

RMSE: 4.171568164713711 MAPE: 2.816802903967086 R2: 0.9384670533581013 MAE: 3.0070179219906454
